In [1]:
import gc
import os
from pathlib import Path
import random
import sys
from tqdm import tqdm
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from IPython.core.display import display, HTML

# --- plotly ---
from plotly import tools, subplots
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
import plotly.io as pio
pio.templates.default = "plotly_dark"

# --- models ---
from sklearn import preprocessing
from sklearn.model_selection import KFold
import lightgbm as lgb
import xgboost as xgb
import catboost as cb

# --- setup ---
pd.set_option('max_columns', 50)

## Outline

- [載入資料](#載入資料)
    - [資料前處理](#資料前處理)
- [綜觀全球疫情走勢](#綜觀全球疫情走勢)
- [begin](#begin-------------------------------------------------------)
    - [預測各國疫情走勢](#預測各國疫情走勢)

## 載入資料

使用到的資料集: [COVID-19/csse_covid_19_data/csse_covid_19_time_series/](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series)

- 全球
    - `confirmed_global_df` : dataframe，儲存從 2020/1/22 到 2020/5/30 各國確診人數

    - `deaths_global_df` : dataframe，儲存從 2020/1/22 到 2020/5/30 各國死亡人數

    - `recovered_global_df` : dataframe，儲存從 2020/1/22 到 2020/5/30 各國康復人數
- 美國
    - `confirmed_us_df` : dataframe，儲存從 2020/1/22 到 2020/5/30 美國確診人數
    
    - `deaths_us_df` : dataframe，儲存從 2020/1/22 到 2020/5/30 美國死亡人數

In [2]:
import requests

for filename in ['time_series_covid19_confirmed_global.csv',
                 'time_series_covid19_deaths_global.csv',
                 'time_series_covid19_recovered_global.csv',
                 'time_series_covid19_confirmed_US.csv',
                 'time_series_covid19_deaths_US.csv']:
    print(f'Downloading {filename}')
    url = f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/{filename}'
    myfile = requests.get(url)
    open(filename, 'wb').write(myfile.content)

confirmed_global_df = pd.read_csv('time_series_covid19_confirmed_global.csv')
deaths_global_df = pd.read_csv('time_series_covid19_deaths_global.csv')
recovered_global_df = pd.read_csv('time_series_covid19_recovered_global.csv')

### 資料前處理

變更日期的格式，由 mm/dd/yy 改成 yy-mm-dd

In [3]:
def _convert_date_str(df):
    try:
        df.columns = list(df.columns[:4]) + [datetime.strptime(d, "%m/%d/%y").date().strftime("%Y-%m-%d") for d in df.columns[4:]]
    except:
        print('_convert_date_str failed with %y, try %Y')
        df.columns = list(df.columns[:4]) + [datetime.strptime(d, "%m/%d/%Y").date().strftime("%Y-%m-%d") for d in df.columns[4:]]

In [4]:
_convert_date_str(confirmed_global_df)
_convert_date_str(deaths_global_df)
_convert_date_str(recovered_global_df)

將鑽石公主號的資料移除，約旦河西岸和加薩走廊的資料包含負值，所以一併移除

In [5]:
# Filter out problematic data points (The West Bank and Gaza had a negative value, cruise ships were associated with Canada, etc.)
removed_states = "Recovered|Grand Princess|Diamond Princess"
removed_countries = "US|The West Bank and Gaza"

confirmed_global_df.rename(columns={"Province/State": "Province_State", "Country/Region": "Country_Region"}, inplace=True)
deaths_global_df.rename(columns={"Province/State": "Province_State", "Country/Region": "Country_Region"}, inplace=True)
recovered_global_df.rename(columns={"Province/State": "Province_State", "Country/Region": "Country_Region"}, inplace=True)

confirmed_global_df = confirmed_global_df[~confirmed_global_df["Province_State"].replace(np.nan, "nan").str.match(removed_states)]
deaths_global_df    = deaths_global_df[~deaths_global_df["Province_State"].replace(np.nan, "nan").str.match(removed_states)]
recovered_global_df = recovered_global_df[~recovered_global_df["Province_State"].replace(np.nan, "nan").str.match(removed_states)]

confirmed_global_df = confirmed_global_df[~confirmed_global_df["Country_Region"].replace(np.nan, "nan").str.match(removed_countries)]
deaths_global_df    = deaths_global_df[~deaths_global_df["Country_Region"].replace(np.nan, "nan").str.match(removed_countries)]
recovered_global_df = recovered_global_df[~recovered_global_df["Country_Region"].replace(np.nan, "nan").str.match(removed_countries)]

將所有日期合併到同一欄位，該日期的累積人數合併到另一欄位

In [6]:
confirmed_global_melt_df = confirmed_global_df.melt(
    id_vars=['Country_Region', 'Province_State', 'Lat', 'Long'], value_vars=confirmed_global_df.columns[4:], var_name='Date', value_name='ConfirmedCases')

deaths_global_melt_df = deaths_global_df.melt(
    id_vars=['Country_Region', 'Province_State', 'Lat', 'Long'], value_vars=confirmed_global_df.columns[4:], var_name='Date', value_name='Deaths')

recovered_global_melt_df = deaths_global_df.melt(
    id_vars=['Country_Region', 'Province_State', 'Lat', 'Long'], value_vars=confirmed_global_df.columns[4:], var_name='Date', value_name='Recovered')

recovered_global_melt_df.head()

,Country_Region,Province_State,Lat,Long,Date,Recovered
0,Afghanistan,NaN,33.0000,65.0000,2020-01-22,0
1,Albania,NaN,41.1533,20.1683,2020-01-22,0
2,Algeria,NaN,28.0339,1.6596,2020-01-22,0
3,Andorra,NaN,42.5063,1.5218,2020-01-22,0
4,Angola,NaN,-11.2027,17.8739,2020-01-22,0


In [7]:
train = confirmed_global_melt_df.merge(deaths_global_melt_df, on=['Country_Region', 'Province_State', 'Lat', 'Long', 'Date'])
train = train.merge(recovered_global_melt_df, on=['Country_Region', 'Province_State', 'Lat', 'Long', 'Date'])

In [8]:
# --- US ---
confirmed_us_df = pd.read_csv('time_series_covid19_confirmed_US.csv')
deaths_us_df = pd.read_csv('time_series_covid19_deaths_US.csv')

# 丟掉不須用到的欄位
confirmed_us_df.drop(['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Combined_Key'], inplace=True, axis=1)
deaths_us_df.drop(['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Combined_Key', 'Population'], inplace=True, axis=1)

# 將 Long_ 欄位改名為 Long
confirmed_us_df.rename({'Long_': 'Long'}, axis=1, inplace=True)
deaths_us_df.rename({'Long_': 'Long'}, axis=1, inplace=True)

# 變更日期的格式，由 mm/dd/yy 改成 yy-mm-dd
_convert_date_str(confirmed_us_df)
_convert_date_str(deaths_us_df)

# clean
# 丟掉不屬於美國的地區
confirmed_us_df = confirmed_us_df[~confirmed_us_df.Province_State.str.match("Diamond Princess|Grand Princess|Recovered|Northern Mariana Islands|American Samoa")]
deaths_us_df = deaths_us_df[~deaths_us_df.Province_State.str.match("Diamond Princess|Grand Princess|Recovered|Northern Mariana Islands|American Samoa")]

# --- Aggregate by province state ---
#confirmed_us_df.groupby(['Country_Region', 'Province_State'])
confirmed_us_df = confirmed_us_df.groupby(['Country_Region', 'Province_State']).sum().reset_index()
deaths_us_df = deaths_us_df.groupby(['Country_Region', 'Province_State']).sum().reset_index()

# remove lat, long.
confirmed_us_df.drop(['Lat', 'Long'], inplace=True, axis=1)
deaths_us_df.drop(['Lat', 'Long'], inplace=True, axis=1)

# 合併日期
confirmed_us_melt_df = confirmed_us_df.melt(
    id_vars=['Country_Region', 'Province_State'], value_vars=confirmed_us_df.columns[2:], var_name='Date', value_name='ConfirmedCases')
deaths_us_melt_df = deaths_us_df.melt(
    id_vars=['Country_Region', 'Province_State'], value_vars=deaths_us_df.columns[2:], var_name='Date', value_name='Deaths')

# 將美國確診及死亡的資料合併
train_us = confirmed_us_melt_df.merge(deaths_us_melt_df, on=['Country_Region', 'Province_State', 'Date'])

In [9]:
train = pd.concat([train, train_us], axis=0, sort=False)

train_us.rename({'Country_Region': 'country', 'Province_State': 'province', 'Date': 'date', 'ConfirmedCases': 'confirmed', 'Deaths': 'fatalities'}, axis=1, inplace=True)
train_us['country_province'] = train_us['country'].fillna('') + '/' + train_us['province'].fillna('')

In [10]:
train.head()

,Country_Region,Province_State,Lat,Long,Date,ConfirmedCases,Deaths,Recovered
0,Afghanistan,NaN,33.0000,65.0000,2020-01-22,0,0,0.0
1,Albania,NaN,41.1533,20.1683,2020-01-22,0,0,0.0
2,Algeria,NaN,28.0339,1.6596,2020-01-22,0,0,0.0
3,Andorra,NaN,42.5063,1.5218,2020-01-22,0,0,0.0
4,Angola,NaN,-11.2027,17.8739,2020-01-22,0,0,0.0


In [11]:
train.rename({'Country_Region': 'country', 'Province_State': 'province', 'Id': 'id', 'Date': 'date', 'ConfirmedCases': 'confirmed', 'Deaths': 'fatalities', 'Recovered': 'recovered'}, axis=1, inplace=True)
train['country_province'] = train['country'].fillna('') + '/' + train['province'].fillna('')

# test.rename({'Country_Region': 'country', 'Province_State': 'province', 'Id': 'id', 'Date': 'date', 'ConfirmedCases': 'confirmed', 'Fatalities': 'fatalities'}, axis=1, inplace=True)
# test['country_province'] = test['country'].fillna('') + '/' + test['province'].fillna('')
train.head()

,country,province,Lat,Long,date,confirmed,fatalities,recovered,country_province
0,Afghanistan,NaN,33.0000,65.0000,2020-01-22,0,0,0.0,Afghanistan/
1,Albania,NaN,41.1533,20.1683,2020-01-22,0,0,0.0,Albania/
2,Algeria,NaN,28.0339,1.6596,2020-01-22,0,0,0.0,Algeria/
3,Andorra,NaN,42.5063,1.5218,2020-01-22,0,0,0.0,Andorra/
4,Angola,NaN,-11.2027,17.8739,2020-01-22,0,0,0.0,Angola/


## 綜觀全球疫情走勢

觀察 `ww_df` 的各個屬性

In [12]:
ww_df = train.groupby('date')[['confirmed', 'fatalities']].sum().reset_index()
ww_df['new_case'] = ww_df['confirmed'] - ww_df['confirmed'].shift(1)
ww_df['growth_factor'] = ww_df['new_case'] / ww_df['new_case'].shift(1)
ww_df.tail()

,date,confirmed,fatalities,new_case,growth_factor
139,2020-06-09,7242497,411455,123335.0,1.190320
140,2020-06-10,7376138,417060,133641.0,1.083561
141,2020-06-11,7514529,421455,138391.0,1.035543
142,2020-06-12,7632607,425388,118078.0,0.853220
143,2020-06-13,7766757,429730,134150.0,1.136113


觀察 `ww_melt_df` 的各個屬性

In [13]:
ww_melt_df = pd.melt(ww_df, id_vars=['date'], value_vars=['confirmed', 'fatalities', 'new_case'])
ww_melt_df

,date,variable,value
0,2020-01-22,confirmed,555.0
1,2020-01-23,confirmed,654.0
2,2020-01-24,confirmed,941.0
3,2020-01-25,confirmed,1434.0
4,2020-01-26,confirmed,2118.0
...,...,...,...
427,2020-06-09,new_case,123335.0
428,2020-06-10,new_case,133641.0
429,2020-06-11,new_case,138391.0
430,2020-06-12,new_case,118078.0


### 全球確診/死亡案例 (折線圖)

- 2020/4/2 確診人數突破 1M，且死亡人數為 52K
- 2020/5/1 確診人數突破 3.3M，且死亡人數為 238K
- **好消息! 每日新增確診案例曲線從 2020/4/4 開始趨於平緩至今**

In [14]:
fig = px.line(ww_melt_df, x="date", y="value", color='variable', 
              title="Worldwide Confirmed/Death Cases Over Time")
fig.show()

### 全球確診/死亡案例 (折線圖) (取log)

- 比較 2020/3 初和 2020/3 底，確診案例成長率的上升速度略為增加

In [15]:
fig = px.line(ww_melt_df, x="date", y="value", color='variable',
              title="Worldwide Confirmed/Death Cases Over Time (Log scale)",
             log_y=True)
fig.show()

### 全球死亡率 (折線圖)

- 可以明顯看到，死亡率在 2020/5 開始下降

In [16]:
ww_df['mortality'] = ww_df['fatalities'] / ww_df['confirmed']

fig = px.line(ww_df, x="date", y="mortality", 
              title="Worldwide Mortality Rate Over Time")
fig.show()

列出有多少國家位於何種確診案例的數量等級

In [17]:
country_df = train.groupby(['date', 'country'])[['confirmed', 'fatalities']].sum().reset_index()
target_date = country_df['date'].max()

print('Date: ', target_date)
for i in [1, 10, 100, 1000, 10000]:
    n_countries = len(country_df.query('(date == @target_date) & confirmed > @i'))
    print(f'{n_countries} countries have more than {i} confirmed cases')

Date:  2020-06-13
188 countries have more than 1 confirmed cases
184 countries have more than 10 confirmed cases
163 countries have more than 100 confirmed cases
120 countries have more than 1000 confirmed cases
59 countries have more than 10000 confirmed cases


列出所有國家

In [18]:
countries = country_df['country'].unique()
print(f'{len(countries)} countries are in dataset:\n{countries}')

188 countries are in dataset:
['Afghanistan' 'Albania' 'Algeria' 'Andorra' 'Angola'
 'Antigua and Barbuda' 'Argentina' 'Armenia' 'Australia' 'Austria'
 'Azerbaijan' 'Bahamas' 'Bahrain' 'Bangladesh' 'Barbados' 'Belarus'
 'Belgium' 'Belize' 'Benin' 'Bhutan' 'Bolivia' 'Bosnia and Herzegovina'
 'Botswana' 'Brazil' 'Brunei' 'Bulgaria' 'Burkina Faso' 'Burma' 'Burundi'
 'Cabo Verde' 'Cambodia' 'Cameroon' 'Canada' 'Central African Republic'
 'Chad' 'Chile' 'China' 'Colombia' 'Comoros' 'Congo (Brazzaville)'
 'Congo (Kinshasa)' 'Costa Rica' "Cote d'Ivoire" 'Croatia' 'Cuba' 'Cyprus'
 'Czechia' 'Denmark' 'Diamond Princess' 'Djibouti' 'Dominica'
 'Dominican Republic' 'Ecuador' 'Egypt' 'El Salvador' 'Equatorial Guinea'
 'Eritrea' 'Estonia' 'Eswatini' 'Ethiopia' 'Fiji' 'Finland' 'France'
 'Gabon' 'Gambia' 'Georgia' 'Germany' 'Ghana' 'Greece' 'Grenada'
 'Guatemala' 'Guinea' 'Guinea-Bissau' 'Guyana' 'Haiti' 'Holy See'
 'Honduras' 'Hungary' 'Iceland' 'India' 'Indonesia' 'Iran' 'Iraq'
 'Ireland' 'Israel'

### 目前確診案例前 30 國家 (折線圖)

In [19]:
country_df = train.groupby(['date', 'country'])[['confirmed', 'fatalities']].sum().reset_index()
top_country_df = country_df.query('(date == @target_date) & (confirmed > 1000)').sort_values('confirmed', ascending=False)

In [20]:
shapes = []
for i in (20, 40, 60):
    shapes.append({'type': 'line', 'xref': 'x', 'yref': 'y', 'x0': i, 'y0': 0, 'x1': i, 'y1': 1})
layout = go.Layout(shapes = shapes)

## begin-------------------------------------------------------

In [21]:
from datetime import timedelta

# plot function
def plot_by_country(country_str, events, anno_pos = 0.5, anno_angle = -90):
    country_data = pd.DataFrame()
    for i in range(len(country_df)):
        if country_df['country'][i] == country_str:
            country_data = country_data.append(country_df.iloc[i])
    country_data.index = np.arange(len(country_data))
    fig = px.line(country_data,
                  x='date', y='confirmed', color='country',
                  title=f'Confirmed Cases of {country_str} till {target_date}',
                  height = 500)
    ymax = country_data['confirmed'][len(country_data) - 1]
    for event in events:
        fig.add_trace(go.Scatter(x=['2020-' + event[0], '2020-' + event[0]], y=[0, ymax], name=event[1],
                                 mode = 'lines', line=dict(dash='dash', color=(event[2]))))
        yesterday = (datetime.strptime('2020-' + event[0], "%Y-%m-%d").date() - timedelta(days=1)).strftime("%Y-%m-%d")
        fig.add_annotation(x=yesterday, y=ymax * anno_pos, text=event[0] + ' ' + event[1],
                           showarrow=False, textangle=anno_angle, bgcolor = '#101010')
    fig.show()
    
# plot function (log)
def plot_by_country2(country_str, events, anno_pos = 0.5, anno_angle = -90):
    country_data = pd.DataFrame()
    for i in range(len(country_df)):
        if country_df['country'][i] == country_str:
            country_data = country_data.append(country_df.iloc[i])
    country_data.index = np.arange(len(country_data))
    country_data['confirmed'] = country_data['confirmed'].apply(lambda x : np.log10(x + 1))
    fig = px.line(country_data,
                  x='date', y='confirmed', color='country',
                  title=f'Confirmed Cases of {country_str} till {target_date} (log)',
                  height = 500)
    ymax = country_data['confirmed'][len(country_data) - 1]
    for event in events:
        fig.add_trace(go.Scatter(x=['2020-' + event[0], '2020-' + event[0]], y=[0, ymax], name=event[1],
                                 mode = 'lines', line=dict(dash='dash', color=(event[2]))))
        yesterday = (datetime.strptime('2020-' + event[0], "%Y-%m-%d").date() - timedelta(days=1)).strftime("%Y-%m-%d")
        fig.add_annotation(x=yesterday, y=ymax * anno_pos, text=event[0] + ' ' + event[1],
                           showarrow=False, textangle=anno_angle, bgcolor = '#101010')
    fig.show()

# plot function (log slope)
def plot_by_country3(country_str, events, anno_pos = 0.5, anno_angle = -90):
    country_data = pd.DataFrame()
    for i in range(len(country_df)):
        if country_df['country'][i] == country_str:
            country_data = country_data.append(country_df.iloc[i])
    country_data.index = np.arange(len(country_data))
    country_data['confirmed'] = country_data['confirmed'].apply(lambda x : np.log10(x + 1))
    slope = [0]
    for i in range(1, len(country_data)):
        slope.append(country_data['confirmed'][i] - country_data['confirmed'][i - 1])
    country_data['slope'] = slope
    fig = px.line(country_data,
                  x='date', y='slope', color='country',
                  title=f'Confirmed Cases of {country_str} till {target_date} (log slope)',
                  height = 500)
    ymax = country_data['slope'][len(country_data) - 1]
    line_y = anno_pos*0.5 if anno_pos > 1 else 0.5
    for event in events:
        fig.add_trace(go.Scatter(x=['2020-' + event[0], '2020-' + event[0]], y=[0, line_y], name=event[1],
                                 mode = 'lines', line=dict(dash='dash', color=(event[2]))))
        yesterday = (datetime.strptime('2020-' + event[0], "%Y-%m-%d").date() - timedelta(days=1)).strftime("%Y-%m-%d")
        fig.add_annotation(x=yesterday, y=float(0.5 * anno_pos), text=event[0] + ' ' + event[1],
                           showarrow=False, textangle=anno_angle, bgcolor = '#101010')
    fig.show()

### 台灣

In [22]:
def gen_random_color(min_value=0, max_value=256) -> str:
    """Generate random color for plotly"""
    r, g, b = np.random.randint(min_value, max_value, 3)
    return f'rgb({r},{g},{b})'  

color_1 = gen_random_color(min_value=20) # 限制境外移入 (病例)
color_2 = gen_random_color(min_value=20) # 擴大篩檢 (列出隱藏病例)
color_3 = gen_random_color(min_value=20) # 口罩禁止出口 (人民開始意識到疫情嚴重性)
color_4 = gen_random_color(min_value=20) # 封城/宵禁/禁足
color_5 = gen_random_color(min_value=20) # 保持社交距離/強制戴口罩

events = [['02-11', '事件1', color_1],
          ['03-22', '事件2', color_2]]
plot_by_country('Taiwan*', events)
plot_by_country2('Taiwan*', events, 0.4, -60)
plot_by_country3('Taiwan*', events, 0.75, -75)

### 美國

In [23]:
events = [['02-11', '事件', color_1]]
plot_by_country('US', events)

### 印度

In [24]:
events = [['03-09', '開始封鎖邊境', 'rgb(200, 200, 200)'],
    ['03-19', '禁止口罩出口', 'rgb(200, 200, 200)'],
    ['03-25', '全國封鎖', 'rgb(200, 200, 200)'],
    ['04-07', '強制戴口罩', 'rgb(200, 200, 200)'],
    ['05-11', '擴大篩檢無症狀者', 'rgb(200, 200, 200)']
]
plot_by_country('India', events)
plot_by_country2('India', events)
plot_by_country3('India', events)

### 英國

In [35]:
events = [
    ['03-23', 'lockdown', 'rgb(200, 200, 200)'],
    ['04-10', '從特定地區回國者須居家隔離', 'rgb(200, 200, 200)'],
    ['05-22', '對進入英國的人的14天自我隔離', 'rgb(200, 200, 200)'],
    ['06-08', '所有入境者一律都要隔離14天', 'rgb(200, 200, 200)'],
    ['04-02', '提供更高的每日病毒檢測數量', 'rgb(200, 200, 200)'],
    ['04-28', '擴大篩檢65歲以上老人及出外工作的人', 'rgb(200, 200, 200)']
]
plot_by_country('United Kingdom', events)
plot_by_country2('United Kingdom', events)
plot_by_country3('United Kingdom', events)

### 俄羅斯

In [25]:
events = [['02-20', '禁止中國公民入境', color_1],
          ['03-01', '暫時禁止出口醫用口罩', color_3],
          ['03-16', '限制歐洲航班', color_1],
          ['03-30', '封城', color_4],]

plot_by_country('Russia', events)

### 日本

In [34]:
events = [['02-01', '限制湖北省相關旅客及所有外國籍患者入境', 'rgb(200, 200, 200)'],
    ['02-13', '允許政府在入境前實施隔離及停留措施', 'rgb(200, 200, 200)'],
    ['02-26', '請求活動取消或延期', 'rgb(200, 200, 200)'],
    ['02-27', '請求全國學校停課', 'rgb(200, 200, 200)'],
    ['03-04', '病毒檢測加入公費保險範圍', 'rgb(200, 200, 200)'],
    ['03-24', '宣布2020東京奧運延期舉行', 'rgb(200, 200, 200)'],
]
plot_by_country('Japan', events)
plot_by_country2('Japan', events)
plot_by_country3('Japan', events)

### 南韓

In [26]:
events = [['02-11', '事件', color_1]]
plot_by_country('Korea, South', events)

## 預測各國疫情走勢

In [27]:
country_df['date'] = pd.to_datetime(country_df['date'])
def sigmoid(t, M, beta, alpha, offset=0):
    alpha += offset
    return M / (1 + np.exp(-beta * (t - alpha)))

def error(x, y, params):
    M, beta, alpha = params
    y_pred = sigmoid(x, M, beta, alpha)

    # apply weight, latest number is more important than past.
    weight = np.arange(len(y_pred)) ** 2
    loss_mse = np.mean((y_pred - y) ** 2 * weight)
    return loss_mse

def gen_random_color(min_value=0, max_value=256) -> str:
    """Generate random color for plotly"""
    r, g, b = np.random.randint(min_value, max_value, 3)
    return f'rgb({r},{g},{b})'

In [28]:
# train and predict

# 請注意模型預設的感染人數必須超過 1000，並且在感染人數超過1000後才開始當作 train data，
# 隨便放入確診數低於 1000 的國家會直接噴 Error !!!
# 後來透過參數列，改成可以指定國家和最低確診人數
# 使用的模型為 scipy.optimize.minimize (非線性的優化?)
# m, beta, alpha 的意義需要更深入釐清

def fit_sigmoid(target_countries, least_confirmed = 1000, exclude_days=0):
    target_country_df_list = []
    pred_df_list = []
    for target_country in target_countries:
        print('target_country', target_country)
        # --- Train ---
        target_country_df = country_df.query('country == @target_country')

        #train_start_date = target_country_df['date'].min()
        train_start_date = target_country_df.query(f'confirmed > {least_confirmed}')['date'].min()
        train_end_date = pd.to_datetime(target_date) - pd.Timedelta(f'{exclude_days} days')
        target_date_df = target_country_df.query('(date >= @train_start_date) & (date <= @train_end_date)')
        if len(target_date_df) <= 7:
            print('WARNING: the data is not enough, use 7 more days...')
            train_start_date -= pd.Timedelta('7 days')
            target_date_df = target_country_df.query('(date >= @train_start_date) & (date <= @train_end_date)')

        confirmed = target_date_df['confirmed'].values
        x = np.arange(len(confirmed))

        lossfun = lambda params: error(x, confirmed, params)
        res = sp.optimize.minimize(lossfun, x0=[np.max(confirmed) * 5, 0.04, 2 * len(confirmed) / 3.], method='nelder-mead')
        M, beta, alpha = res.x
        # sigmoid_models[key] = (M, beta, alpha)
        # np.clip(sigmoid(list(range(len(data), len(data) + steps)), M, beta, alpha), 0, None).astype(int)

        # --- Pred ---
        pred_start_date = target_country_df['date'].min()
        pred_end_date = pd.to_datetime('2020-07-01')
        days = int((pred_end_date - pred_start_date) / pd.Timedelta('1 days'))
        #print('pred start', pred_start_date, 'end', pred_end_date, 'days', days)

        x = np.arange(days)
        offset = (train_start_date - pred_start_date) / pd.Timedelta('1 days')
        print('train_start_date', train_start_date, 'offset', offset, 'M =', M, 'beta =', beta, 'alpha =', alpha)
        y_pred = sigmoid(x, M, beta, alpha, offset=offset)
        # target_country_df['confirmed_pred'] = y_pred

        all_dates = [pred_start_date + np.timedelta64(x, 'D') for x in range(days)]
        pred_df = pd.DataFrame({
            'date': all_dates,
            'country': target_country,
            'confirmed_pred': y_pred,
        })

        target_country_df_list.append(target_country_df)
        pred_df_list.append(pred_df)
    return target_country_df_list, pred_df_list

In [29]:
# plot predictions

def plot_sigmoid_fitting(target_countries, target_country_df_list, pred_df_list, title=''):
    fig = go.Figure()
    for i in range(len(target_countries)):
        target_country = target_countries[i]
        target_country_df = target_country_df_list[i]
        pred_df = pred_df_list[i]
        color = gen_random_color(min_value=20)
        # Prediction
        fig.add_trace(go.Scatter(
            x=pred_df['date'], y=pred_df['confirmed_pred'],
            name=f'{target_country}_pred',
            line=dict(color=color, dash='dash')
        ))

        # Ground truth
        fig.add_trace(go.Scatter(
            x=target_country_df['date'], y=target_country_df['confirmed'],
            mode='markers', name=f'{target_country}_actual',
            line=dict(color=color),
        ))
    fig.update_layout(
        title=title, xaxis_title='Date', yaxis_title='Confirmed cases')
    fig.show()

In [30]:
target_countries = ['Taiwan*']
target_country_df_list, pred_df_list = fit_sigmoid(target_countries, least_confirmed = 10, exclude_days=0)
plot_sigmoid_fitting(target_countries, target_country_df_list, pred_df_list, title='Sigmoid fitting with TW')

target_country Taiwan*
train_start_date 2020-02-04 00:00:00 offset 13.0 M = 439.9521310343762 beta = 0.13178809451978868 alpha = 50.02801965825057


In [31]:
target_countries = ['US', 'Russia']
target_country_df_list, pred_df_list = fit_sigmoid(target_countries, least_confirmed = 1000, exclude_days=0)
plot_sigmoid_fitting(target_countries, target_country_df_list, pred_df_list, title='Sigmoid fitting with all latest data')

target_country US
train_start_date 2020-03-11 00:00:00 offset 49.0 M = 2315296.15985271 beta = 0.048950032221146404 alpha = 54.42374246704651
target_country Russia
train_start_date 2020-03-27 00:00:00 offset 65.0 M = 594938.9625563284 beta = 0.07151198259728042 alpha = 53.47931179780718
